# Load data

In [1]:
import pandas as pd

## Load positive and negative words

In [2]:
neg_words = pd.DataFrame(
    open("../data/negative-words.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)

pos_words = pd.DataFrame(
    open("../data/negative-words.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)

neg_words["label"] = 0
pos_words["label"] = 1


word_df = pd.concat([pos_words, neg_words], ignore_index=True)

#shuffle rows
word_df = word_df.sample(frac=1).reset_index(drop=True)

word_df.head()

,review,label
0,obsolete,0
1,sneak,1
2,pretentiously,1
3,wobbles,0
4,unsure,1


## Load positive and negative reviews

In [3]:
neg_reviews = pd.DataFrame(
    open("../data/negative-reviews.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)

pos_reviews = pd.DataFrame(
    open("../data/positive-reviews.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)
neg_reviews["label"] = 0
pos_reviews["label"] = 1


review_df = pd.concat([neg_reviews, pos_reviews], ignore_index=True)

#shuffle rows
review_df = review_df.sample(frac=1).reset_index(drop=True)

review_df.head()

,review,label
0,Functionality Quality Reputation and Price,1
1,limited amount of paper load,0
2,"Great image quality, Carl Zeiss lens with 5x o...",1
3,cheap,1
4,"clarity, resolution, zoom power",1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)